###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es Unidad de Planificación Rural Agropecuaria - UPRA del Ministerio de Agricultura

In [2]:
import pandas as pd

###Archivo fyc7-sbtz.csv

Contieneinformación sobre la identificación de la frontera agrícola y frontera agrícola condicionada en Colombia, a escala 1: 100.000

In [3]:
df = pd.read_csv("https://www.datos.gov.co/resource/fyc7-sbtz.csv?$limit=500000", dtype={'cod_dane_m': str})
df.head()

,municipio,departamen,cod_depart,cod_dane_m,tipo_front,area_ha,consecutiv
0,Abejorral,Antioquia,5,05002,Condicionada,0.168775,1
1,Abejorral,Antioquia,5,05002,Condicionada,0.427561,2
2,Abejorral,Antioquia,5,05002,Condicionada,0.136877,3
3,Abejorral,Antioquia,5,05002,Condicionada,0.831059,4
4,Abejorral,Antioquia,5,05002,Condicionada,0.003190,5


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455143 entries, 0 to 455142
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   municipio   455143 non-null  object 
 1   departamen  455143 non-null  object 
 2   cod_depart  455143 non-null  int64  
 3   cod_dane_m  455143 non-null  object 
 4   tipo_front  455143 non-null  object 
 5   area_ha     455143 non-null  float64
 6   consecutiv  455143 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 24.3+ MB


In [6]:
df.columns

Index(['municipio', 'departamen', 'cod_depart', 'cod_dane_m', 'tipo_front',
       'area_ha', 'consecutiv'],
      dtype='object')

### Eliminación de columnas irrelevantes para el proyecto

In [8]:
relevant_cols = ['municipio', 'departamen',  'cod_dane_m', 'tipo_front', 'area_ha', 'consecutiv']
agricultural_frontier_minagricultura = df[relevant_cols]
agricultural_frontier_minagricultura.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455143 entries, 0 to 455142
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   municipio   455143 non-null  object 
 1   departamen  455143 non-null  object 
 2   cod_dane_m  455143 non-null  object 
 3   tipo_front  455143 non-null  object 
 4   area_ha     455143 non-null  float64
 5   consecutiv  455143 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 20.8+ MB


### Verificar valores nulos

In [9]:
agricultural_frontier_minagricultura.isnull().sum()

,0
municipio,0
departamen,0
cod_dane_m,0
tipo_front,0
area_ha,0
consecutiv,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [10]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['municipio', 'departamen',  'cod_dane_m', 'tipo_front']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(agricultural_frontier_minagricultura[column].unique())
    print()


Categorías en la columna 'municipio':
['Abejorral' 'Ábrego' 'Abriaquí' ... 'Zipacón' 'Zipaquirá' 'Zona Bananera']

Categorías en la columna 'departamen':
['Antioquia' 'Norte de Santander' 'Meta' 'Chocó' 'Huila' 'Bolívar'
 'Cundinamarca' 'Cesar' 'Santander' 'Caldas' 'Casanare' 'Nariño' 'Caquetá'
 'La Guajira' 'Valle del Cauca' 'Magdalena' 'Cauca' 'Boyacá' 'Tolima'
 'Risaralda' 'Arauca' 'Quindío' 'Córdoba' 'Atlántico' 'Guainía' 'Sucre'
 'Guaviare' 'Vaupés' 'Putumayo' 'Vichada' 'Amazonas'
 'San Andrés Providencia y Santa Catalina']

Categorías en la columna 'cod_dane_m':
['05002' '54003' '05004' ... '25898' '25899' '47980']

Categorías en la columna 'tipo_front':
['Condicionada' 'No condicionada']



- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [11]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%', '"', "'", '/', '\\', '|', '-']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [12]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    agricultural_frontier_minagricultura.loc[:,col] = agricultural_frontier_minagricultura[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

### Verificar que los valores en "cod_dane_m" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [15]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "cod_dane_m" en el df agricultural_frontier_minagricultura

In [16]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
agricultural_frontier_minagricultura.loc[:,'cod_dane_m'] = agricultural_frontier_minagricultura['cod_dane_m'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = agricultural_frontier_minagricultura['cod_dane_m'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

True

In [17]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5]


In [18]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,cod_dane_m,count,percentage
0,5,455143,1.0


In [19]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = agricultural_frontier_minagricultura[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['cod_dane_m']])
    print()

Muestra de registros con longitud 5:
  cod_dane_m
0      05002
1      05002
2      05002
3      05002
4      05002



Nota: el 100% de los condigos del dataset tienen el mismo formato que los cod reales que se encuentran en la bd: string de 5 digitos.

- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [20]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

In [21]:
# Comparar listas de códigos
compare_lists(agricultural_frontier_minagricultura['cod_dane_m'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en agricultural_frontier_minagricultura", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en agricultural_frontier_minagricultura que no están en Códigos de municipios dept_mpios_codes:
{'94663'}


  - Identifcar los registros bajo este código que no corresponde a ningun código real de municipio

In [23]:
cod_wrong = agricultural_frontier_minagricultura[agricultural_frontier_minagricultura['cod_dane_m'] == '94663']
cod_wrong['municipio'].unique()


array(['MAPIRIPANA'], dtype=object)

Mapiripana una poblacion del Guania que en la tabla geolocalizada de municipios de la bd figura como corregimiento del municipio BarrancoMinas de Guania. Por lo tanto reemplazamos este codigo por el de Municipio al que pertenece

In [24]:
# Listado de municipios de Guania
dept_mpios_codes[dept_mpios_codes['dept_name'] == 'GUAINIA']

,dept_mpio_code,dept_name,mupio_name
1092,94001,GUAINIA,INIRIDA
1093,94343,GUAINIA,BARRANCOMINAS
1094,94883,GUAINIA,SAN FELIPE
1095,94884,GUAINIA,PUERTO COLOMBIA
1096,94885,GUAINIA,LA GUADALUPE
1097,94886,GUAINIA,CACAHUAL
1098,94887,GUAINIA,PANA PANA
1099,94888,GUAINIA,MORICHAL


In [25]:
agricultural_frontier_minagricultura.loc[:,'cod_dane_m'] = agricultural_frontier_minagricultura['cod_dane_m'].replace( '94663', '94343')

- Verificar nuevamente que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [26]:
# Comparar listas de códigos
compare_lists(agricultural_frontier_minagricultura['cod_dane_m'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en agricultural_frontier_minagricultura", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en agricultural_frontier_minagricultura que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [28]:
# Eliminar columnas que ya no se necesitan
columns_to_drop = ['departamen', 'municipio']
agricultural_frontier_minagricultura = agricultural_frontier_minagricultura.drop(columns=columns_to_drop)

In [29]:
# Adicionar columna para trazabilidad de la fuente
agricultural_frontier_minagricultura['source_id'] = 126

In [31]:
agricultural_frontier_minagricultura.columns

Index(['cod_dane_m', 'tipo_front', 'area_ha', 'consecutiv', 'source_id'], dtype='object')

In [33]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'cod_dane_m': 'dane_code',
    'tipo_front': 'frontier_type',
    'area_ha': 'area_ha',
    'consecutiv': 'consecutive',
    'source_id': 'source_id'
}

# Renombrar las columnas
agricultural_frontier_minagricultura.rename(columns=translation_map, inplace=True)

In [34]:
#Estructura final del dataset a integrar a la base de datos
agricultural_frontier_minagricultura.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455143 entries, 0 to 455142
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   dane_code      455143 non-null  object 
 1   frontier_type  455143 non-null  object 
 2   area_ha        455143 non-null  float64
 3   consecutive    455143 non-null  int64  
 4   source_id      455143 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 17.4+ MB


## Salvar en archivo csv en el drive

In [35]:
# Guardar en archivos CSV en el drive
agricultural_frontier_minagricultura.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/agricultural_frontier_minagricultura.csv', index=False)